In [1]:
#1モデルの指定 PyTorchバージョン
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89990 sha256=346d27510a36e5e09eb7297645f02e054f3b6785a009332cc157f632ba71ae02
  Stored in directory: /root/.cache/pip/wheels/fc/5d/45/34fe9945d5e45e261134e72284395be36c2d4828af38e2b0fe
Successfully built lit
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.

In [1]:
!pip install "numpy<2"

In [4]:
#2モデルの指定xformersバージョン
!pip install xformers==0.0.22


In [7]:
#main 環境設置
!pip install git+https://github.com/deepseek-ai/deepseek-vl2.git

  Cloning https://github.com/deepseek-ai/deepseek-vl2.git to /tmp/pip-req-build-j_5h1k8x
  Running command git clone --filter=blob:none --quiet https://github.com/deepseek-ai/deepseek-vl2.git /tmp/pip-req-build-j_5h1k8x
  Resolved https://github.com/deepseek-ai/deepseek-vl2.git to commit ef9f91e2b6426536b83294c11742c27be66361b1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 109.2 MB/s eta 0:00:00
  Created wheel for deepseek_vl2: filename=deepseek_vl2-1.0.0-py3-none-any.whl size=72458 sha256=2f5934c4fa73537905df9e8961fca742a83dd521421fa109287ccc7f22bdd88c
  Stored in directory: /tmp/pip-ephem-wheel-cache-bdm8l9nc/wheels/a8/3b/ee/22272a2bf61c51044b87cfd55553724ec298ead3ad2f2dd4c7
Successfull

In [ ]:
#必要環境検査
!pip show transformers || pip install -q transformers && pip show transformers
!pip show accelerate || pip install -q accelerate && pip show accelerate
!pip show deepseek-vl2 || pip install -q git+https://github.com/deepseek-ai/deepseek-vl2.git && pip show deepseek-vl2


Name: transformers
Version: 4.38.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: deepseek_vl2, peft, sentence-transformers
Name: transformers
Version: 4.38.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /

In [1]:
import torch
from google.colab import files
import os, csv, re
from deepseek_vl2.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM
from deepseek_vl2.utils.io import load_pil_images
import ipywidgets as widgets


# ------------------------------
# ✅ OCR モードはdeepseek desu
# ------------------------------
class OCRModel:
    def __init__(self, model_path="deepseek-ai/deepseek-vl2-tiny"):
        self.processor = DeepseekVLV2Processor.from_pretrained(model_path)
        self.tokenizer = self.processor.tokenizer
        self.model = DeepseekVLV2ForCausalLM.from_pretrained(model_path, trust_remote_code=True)
        self.model.config.use_flash_attention = False
        self.model = self.model.to("cuda" if torch.cuda.is_available() else "cpu").eval()

    def predict(self, image_path):
        conversation = [
            {
                "role": "<|User|>",
                "content": """<image>\n
                言語設定：日本語。
                以下の項目だけ答えてください：
                - 国籍（例：ベトナム、韓国、中国など）
                - 氏名（氏名は原文の通り、たとえ英語でもカタカナに変換しないでください。）
                - 性別（男 または 女）
                - 生年月日
                - 住居地（都道府県から）
                - 在留資格（例：留学、家族滞在など）
                - 在留カード番号（右上の番号は在留カード番号です）
                - 在留期間(PERIOD OF STAY):X年X月 (XXXX年XX月XX日)
                それ以外の説明や解説は不要です。""",
                "images": [image_path],
            },
            {"role": "<|Assistant|>", "content": ""},
        ]

        pil_images = load_pil_images(conversation)
        inputs = self.processor(conversations=conversation, images=pil_images, force_batchify=True)
        inputs = inputs.to(self.model.device)

        inputs_dict = dict(inputs)

        # ✅  float32　だけ
        for k, v in inputs_dict.items():
            if isinstance(v, torch.Tensor) and k in ["pixel_values", "images"]:
                inputs_dict[k] = v.to(torch.float32)

        inputs_embeds = self.model.prepare_inputs_embeds(**inputs_dict)

        outputs = self.model.generate(
            inputs_embeds=inputs_embeds,
            attention_mask=inputs_dict.get("attention_mask"),
            pad_token_id=self.tokenizer.eos_token_id,
            bos_token_id=self.tokenizer.bos_token_id,
            eos_token_id=self.tokenizer.eos_token_id,
            max_new_tokens=500,
            do_sample=False,
            use_cache=True,
        )

        return self.tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)

# ------------------------------
# ✅ 文字列
# ------------------------------
def extract_fields(text):
    def extract_multiline_address(text):
        match = re.search(r"住居地[:：]?\s*(.+)", text)
        return match.group(1).strip() if match else ""

    def extract_stay_period(text):
        pattern = r"在留期間(?:\(PERIOD OF STAY\))?[:：]?\s*(\d+年(?:\d+月)?)\s*[（(](\d{4}年\d{1,2}月\d{1,2}日)[）)]"
        match = re.search(pattern, text)
        return (match.group(1), match.group(2)) if match else ("", "")

    def extract_card_number(text):
        match = re.search(r"在留カード番号[:：]?\s*([A-Z0-9]+)", text)
        return match.group(1) if match else ""

    return {
        "国籍": re.search(r"国籍[:：]?\s*([^\s\n]+)", text).group(1) if re.search(r"国籍[:：]?\s*([^\s\n]+)", text) else "",
        "氏名": re.search(r"氏名[:：]?\s*(.+)", text).group(1).strip() if re.search(r"氏名[:：]?\s*(.+)", text) else "",
        "性別": re.search(r"性別[:：]?\s*(男|女)", text).group(1) if re.search(r"性別[:：]?\s*(男|女)", text) else "",
        "生年月日": re.search(r"生年月日[:：]?\s*(\d{4}年\d{1,2}月\d{1,2}日)", text).group(1) if re.search(r"生年月日[:：]?\s*(\d{4}年\d{1,2}月\d{1,2}日)", text) else "",
        "住居地": extract_multiline_address(text),
        "在留資格": re.search(r"在留資格[:：]?\s*(\S+)", text).group(1) if re.search(r"在留資格[:：]?\s*(\S+)", text) else "",
        "在留カード番号": extract_card_number(text),
        "在留期間_期間": extract_stay_period(text)[0],
        "在留期間_満了日": extract_stay_period(text)[1],
    }

# ------------------------------
# ✅ 保存 CSV
# ------------------------------
def save_to_csv(data_list, path):
    keys = ["学籍番号"] + [k for k in data_list[0].keys() if k != "学籍番号"]
    with open(path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=keys)
        writer.writeheader()
        for row in data_list:
            writer.writerow(row)

# ------------------------------
# ✅ 一つファイル
# ------------------------------
# def process_single():
#     uploaded = files.upload()
#     for filename in uploaded.keys():
#         result = ocr.predict(filename)
#         fields = extract_fields(result)
#         fields["学籍番号"] = os.path.splitext(os.path.basename(filename))[0]
#         save_to_csv([fields], "result_single.csv")
#         files.download("result_single.csv")
# def process_single():
#     uploaded = files.upload()
#     for filename in uploaded.keys():
#         result = ocr.predict(filename)
#         fields = extract_fields(result)
#         fields["学籍番号"] = os.path.splitext(os.path.basename(filename))[0]

#         # ✅ Step 1: 显示OCR提取结果
#         print("\n📄 抽出された項目:")
#         for key, value in fields.items():
#             print(f"{key}: {value}")

#         # ✅ Step 2: 创建确认按钮
#         confirm_button = widgets.ToggleButtons(
#             options=["はい、CSV保存する", "いいえ、保存しない"],
#             description="保存しますか？",
#             disabled=False,
#             button_style='',
#             tooltips=["保存する", "保存しない"],
#         )
#         display(confirm_button)

#         # ✅ Step 3: 等待用户确认
#         def on_confirm_change(change):
#             if change["type"] == "change" and change["name"] == "value":
#                 if change["new"] == "はい、CSV保存する":
#                     save_to_csv([fields], "result_single.csv")
#                     files.download("result_single.csv")
#                     print("✅ 保存しました: result_single.csv")
#                 else:
#                     print("❌ 保存しませんでした。")
#                 confirm_button.close()

#         confirm_button.observe(on_confirm_change)
#         break
def process_single():
    uploaded = files.upload()
    for filename in uploaded.keys():
        result = ocr.predict(filename)

        # ✅ Step 1: 显示 OCR 原始全文
        print("\n📝 OCR 抽出された全文:")
        print(result)

        # ✅ Step 2: 字段抽出
        fields = extract_fields(result)
        fields["学籍番号"] = os.path.splitext(os.path.basename(filename))[0]

        # ✅ Step 3: 显示字段结果
        print("\n📄 抽出された項目:")
        for key, value in fields.items():
            print(f"{key}: {value}")

        # ✅ Step 4: 创建确认按钮
        confirm_button = widgets.ToggleButtons(
            options=["はい、CSV保存する", "いいえ、保存しない"],
            description="保存しますか？",
            disabled=False,
            button_style='',
            tooltips=["保存する", "保存しない"],
        )
        display(confirm_button)

        # ✅ Step 5: 等待用户确认是否保存
        def on_confirm_change(change):
            if change["type"] == "change" and change["name"] == "value":
                if change["new"] == "はい、CSV保存する":
                    save_to_csv([fields], "result_single.csv")
                    files.download("result_single.csv")
                    print("✅ 保存しました: result_single.csv")
                else:
                    print("❌ 保存しませんでした。")
                confirm_button.close()

        confirm_button.observe(on_confirm_change)
        break  # 只处理1张图像
# ------------------------------
# ✅ 多数ファイル
# ------------------------------
def process_batch():
    uploaded = files.upload()
    results = []
    for filename in uploaded.keys():
        result = ocr.predict(filename)
        fields = extract_fields(result)
        fields["学籍番号"] = os.path.splitext(os.path.basename(filename))[0]
        results.append(fields)
    save_to_csv(results, "result_batch.csv")
    files.download("result_batch.csv")

# ------------------------------
# ✅ OCRモデル初期化
# ------------------------------
ocr = OCRModel("deepseek-ai/deepseek-vl2-tiny")
print("✅ OCRモデル初期化完了。以下を実行してください：")
print("- process_single()：1枚の画像を認識")
print("- process_batch()：複数画像を一括認識")


Python version is above 3.10, patching the collections module.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Add pad token = ['<｜▁pad▁｜>'] to the tokenizer
<｜▁pad▁｜>:2
Add image token = ['<image>'] to the tokenizer
<image>:128815
Add grounding-related tokens = ['<|ref|>', '<|/ref|>', '<|det|>', '<|/det|>', '<|grounding|>'] to the tokenizer with input_ids
<|ref|>:128816
<|/ref|>:128817
<|det|>:128818
<|/det|>:128819
<|grounding|>:128820
Add chat tokens = ['<|User|>', '<|Assistant|>'] to the tokenizer with input_ids
<|User|>:128821
<|Assistant|>:128822



The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


✅ OCRモデル初期化完了。以下を実行してください：
- process_single()：1枚の画像を認識
- process_batch()：複数画像を一括認識


In [2]:
#単ファイルOCR
process_single()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Saving 22TE429.png to 22TE429 (13).png

📝 OCR 抽出された全文:
国籍：ベトナム
氏名：TO THIEN LONG
性別：男
生年月日：2月5日
住居地：東京都台東区上野1丁目
在留資格：留学
在留カード番号：SA68
在留期間：1年8月（2026年06月01日）
在留期間更新許可：2024年10月01日
 expiration date: Y M D

📄 抽出された項目:
国籍: ベトナム
氏名: TO THIEN LONG
性別: 男
生年月日: 
住居地: 東京都台東区上野1丁目
在留資格: 留学
在留カード番号: SA68
在留期間_期間: 1年8月
在留期間_満了日: 2026年06月01日
学籍番号: 22TE429 (13)


ToggleButtons(description='保存しますか？', options=('はい、CSV保存する', 'いいえ、保存しない'), tooltips=('保存する', '保存しない'), value='は…

❌ 保存しませんでした。


In [4]:
#多数ファイルOCR
process_batch()

Saving 22TE429.png to 22TE429 (15).png
Saving 22TE487.jpg to 22TE487 (5).jpg
Saving 23TE465.png to 23TE465 (1).png
Saving 番号不明.jpg to 番号不明 (2).jpg


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#モード再開
import os
os.kill(os.getpid(), 9)